# Simulating Two Dimentional Heat Equation Using Python And Finite Difference Explicit Method

# Introduction

In this post, we will illustrate how to simulate the heat equation on a two dimensional surface using Python programming language. We use finite difference explit method to solve the heat equation described in the [**previous blog post**](https://shreyaspunacha.github.io/post/heat_equation/).

Here, we will solve the heat equation on a surface having dimensions $L_{x}$ x $L_{y}$ and diffusion coefficient, $D$. We divide the domain into mesh having $nx$ and $ny$ points with the grid spacing $dx$ and $dy$. Therefore, $L_{x}= n_{x} dx$ and $L_{y} = n_{y} dy$. We are seeking the solution for the temperature field $u(x,y)$ for $t \in (0,T]$.

As an initial condition, we use a Guassian heat pulse of the following form centered on the domain. Here, the parameter $\alpha$ controls the width of the Guassian and the function is centered at $(x_{0},y_{0})$.


$$
u(x,y) = e^{-\alpha(x-x_{0})^2 - \alpha (y-y_{0})^2}
$$


Initially, for simplicity, we use the following Dirichlet boundary conditions: $ u(x, 0, t) = u(x, L_{y}, t) = u(0, y, t) = u(L_{x}, y, t) = 0$. Note that these boundary conditions need to applied at each timestep.



# The Python Code

```python
# import packages

import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
```

```python

def Initial_Condition(nx, ny, dx, dy):
    
    """
    Generate a mesh using meshgrid command and then define 
    the Guassian heat pulse as initial condition located at 
    the center of the domain. i.e at the point (nx/2, ny/2).
    
    """ 
    x = np.arange(0, nx, dx) 
    y = np.arange(0, ny, dy) 
    x, y = np.meshgrid(x, y)
    alpha = 0.01
    u0  = np.exp(-alpha*(x-(nx/2))**2-alpha*(y-(ny/2))**2) 
    np.savetxt("Initial_Condition.dat",u0)
    return None
```

```python
def Evolution(u, u_previous, D, dt, dx2, dy2):

    """

    Evolution function using finite difference 
    explicit method. Apply dirichlet initial
    condition here. Note that it applies at
    all timesteps.

    u:              new temperature field
    u_previous:     previous temperature field
    D:              Diffusion constant
    dt:             time step
    dx2:        square of grid spacing along x direction, dx^2
    dy2:        square of grid spacing along x direction, dy^2

    """
    
    # Dirichlet boundary conditions.
    
    u0[0, :] = 0.0
    u0[-1, :] = 0.0
    u0[:, 0] = 0.0
    u0[:, -1] = 0.0

    # Use numpy slicing to vectorise the finite difference scheme.
    
    u[1:-1, 1:-1] = u_previous[1:-1, 1:-1] + \
                    D*dt*( (u_previous[2:, 1:-1] - 2 * u_previous[1:-1, 1:-1] + u_previous[:-2, 1:-1]) / dx2 + \
                    (u_previous[1:-1, 2:] - 2 *u_previous[1:-1, 1:-1] + u_previous[1:-1, :-2]) / dy2 )
    u_previous[:] = u[:]  # Array exchange before next time step 
    return None
```

<div class="alert alert-block alert-info">
Note that we have used numpy slicing to vectorise the finite difference scheme. The equivalent scalar version using for loops is given below. However, it is very slow. 

```python
for i in range(1, nx):
    for j in range(1, ny):
        u[i,j] = u_previous[i,j] + D*dt* ( (u_previous[i+1,j]-2*u_previous[i,j]+u_previous[i-1,j])/dx2 +
                 (u_previous[i,j+1]-2*u_previous[i,j]+u_previous[i,j-1])/dy2 )
```
</div>                  

```python
def Iteration(field, field0, D, dt, dx2, dy2, timesteps, saving_interval):

    """

    Run fixed number of time steps of the heat equation. 
    saving_interval is the image plotting and saving 
    interval whose value can be defined according to 
    convinence.

    """

    for t in range(1, timesteps+1):
        Evolution(field, field0, D, dt, dx2, dy2)
        if t % saving_interval == 0:
            Plot_Field(field, t)
    return None
```

```python
def Read_Fields(filename):

    """
    
    Read the initial temperatue field from file
    
    """

    field = np.loadtxt(filename)
    field0 = field.copy() # Array for field of previous time step
    return field, field0
```

```python
def Plot_Field(field, step):
    """
    
    Function to plot the field at the desired timestep.
    
    """
    plt.gca().clear()
    plt.imshow(field)
    plt.axis("off")
    plt.savefig("heat_{0:03d}.png".format(step))
    plt.clf()
    return None
```

```python
def main(input_file, D, dt, nx, ny, dx, dy, timesteps, saving_interval):


    # Initial condition
    Initial_Condition(nx, ny, dx, dy)

    # Read the initial temperature field from file 
    field, field0 = Read_Fields(input_file)

    # Plot the initial field. i.e at t=0.
    Plot_Field(field, 0)

    # Iterate
    t0 = time.time()
    Iteration(field, field0, D, dt, dx2, dy2, timesteps, saving_interval)
    t1 = time.time()

    # Plot the final field
    Plot_Field(field, timesteps)
    
    # print the time taken to perform the iterations.
    print("Simulations finished in {0} s".format(t1-t0))
    return None
```


```python
if __name__ == "__main__":    
    input_file = "Initial_Condition.dat" 
    D = 20
    nx, ny = 50, 50
    dx, dy = 0.1, 0.1
    dx2 = dx*dx
    dy2 = dy*dy
    # For stability, this is the largest interval possible
    # for the size fo the time step:
    dt = dx2*dy2 / (2*D*(dx2+dy2))
    timesteps = 20000
    saving_interval = 500
    main(input_file, D, dt, nx, ny, dx, dy, timesteps, saving_interval)
```